In [1]:
import pickle
from sage.all import sigma, nth_prime, pari
import ast
import math

from scipy.signal import find_peaks, correlate
from scipy.fft import fft, fftfreq
import numpy as np


def tau(n):
    def step1(n, k):
        return 35*k^4 - 52*k^3*n + 18*k^2*n^2
    def step2(n, k):
        return sigma(k) * sigma(n - k)
    def step3(n):
        ans = 0
        for k in range(1, n):
            ans=ans+step1(n, k) * step2(n, k)
        return 24*ans
    return n^4 * sigma(n) - step3(n)

def tau_prime(k):
            pn=nth_prime(k)
            return tau(pn)

    
def P_matrix(lst):
    lenlist=len(lst)
    columns=[lst]
    for k in range(1,lenlist):
        column=[0]*(k-1)
        column=column+[-k]
        column=flatten(column+[lst])
        column=column+[lst]
        column=column[:lenlist]
        columns+=[column]
    return (matrix(columns)).transpose()



from sage.all import *
precision_bits = 100
x = polygen(QQ)  # Can remain in preamble if all polynomials are in one variable


main_dets=[]
main_j_list=[]
dets=[]  

tau_prime_list = [(k, tau_prime(k)) for k in range(1, 400)]
h_list = [(0, 1)] + tau_prime_list   


j_list=[(0,0)]# <<< this is the deforming code
for k in range(1,53):
    sm=0
    for r in [1..k-1]:
        sm=sm+j_list[r][1]*h_list[k-r][1] 
    j_k=k*h_list[k][1]-sm
    j_list.append((k,j_k))
    main_j_list.append((-1,j_list))
print("undeformed j_list:")
print(j_list)
    
for n in range(2,5):
    print("undeformed",n)
    sliced=j_list[1:n+1] ### undeformed
    inputs=[pair[1] for pair in sliced]
    pl=P_matrix(inputs)
    show(pl)
    det=pl.det()
    dets.append((-1,n,det))
    print((-1,n,det))
    main_dets.append((-1,dets))

undeformed j_list:
[(0, 0), (1, -24), (2, -72), (3, 18810), (4, 518528), (5, 10323516), (6, 34401720), (7, -3989135538), (8, -155766935712), (9, -2992865260284), (10, -17819053859692), (11, 1003061903788624), (12, 42629200194219392), (13, 887368323086397882), (14, 6892985740168908252), (15, -240526862448815484060), (16, -11647363698649213927200), (17, -259657289589751889306000), (18, -2462054998035044872641948), (19, 55685703329232047991048336), (20, 3154243898799515612423138408), (21, 75338156120567569892139003822), (22, 833389948436624377972027401376), (23, -12198334508698806157459655177412), (24, -846700942602570744349975288586400), (25, -21678764514315662032598172517166934), (26, -272086694067516700774544929205162192), (27, 2437537980730937013116397249312332520), (28, 225086801337991985559395131107073147008), (29, 6189508197797528062404657187839688754254), (30, 86502818088520663990377403879139460701180), (31, -402849623279827776392944185677661083969704), (32, -591944643273640657287

[-24  -1]
[-72 -24]

(-1, 2, 504)
undeformed 3


[  -24    -1     0]
[  -72   -24    -2]
[18810   -72   -24]

(-1, 3, 28980)
undeformed 4


[   -24     -1      0      0]
[   -72    -24     -2      0]
[ 18810    -72    -24     -3]
[518528  18810    -72    -24]

(-1, 4, -401856)


In [3]:
from sage.all import (QQ, sigma, nth_prime, matrix, flatten, polygen)
import pickle

Rc.<c> = QQ[]
Rn.<n> = QQ[]
Rd.<d>=QQ[]
Rkk.<kk>=QQ[]

def tau(n):
    def step1(n, k):
        return 35*k^4 - 52*k^3*n + 18*k^2*n^2

    def step2(n, k):
        return sigma(k) * sigma(n - k)

    def step3(n):
        ans = 0
        for k in range(1, n):
            ans += step1(n, k) * step2(n, k)
        return 24*ans

    return n^4 * sigma(n) - step3(n)


def tau_prime(k):
    return tau(nth_prime(k))

def P_matrix(lst):
    L = len(lst)
    columns = [lst]
    for k in range(1, L):
        column = [0]*(k-1) + [-k]
        column = flatten(column + [lst])
        column = column + [lst]
        column = column[:L]
        columns.append(column)
    return matrix(columns).transpose()


            
tau_prime_list=[(k,tau_prime(k)) for k in range(1,300)]
h_list=[[0,1]]+tau_prime_list  
c_polys=[]
c_cffs=[]

for n in range(2,201):
    print("n:",n)
    data=[]
    
    for c in range (1,n+5):
        j_list=[(0,c)]
        
        for k in range(1,n+1):
            sm=0
            
            for r in range(1,k):
                sm+=j_list[r][1]*h_list[k-r][1] 
    
            j_k=k*h_list[k][1]-sm
            j_list.append((k,j_k))
    
        sliced = j_list[:n]
        inputs = [pair[1] for pair in sliced]
        det = P_matrix(inputs).det()
        data.append((c, det))
        
    c_poly = Rc.lagrange_polynomial(data)
    
    lp=list(c_poly)
    lp.reverse()
    c_polys.append((n, lp))
    
    

with open('/Users/barrybrent/data2/run8feb26no3.txt', 'wb') as wfile:
    pickle.dump(c_polys, wfile) # ouput snipped


# set precision
CC = ComplexField(100)                           # [web:13]

# choose a base ring for the coefficients
R.<x> = PolynomialRing(QQ)                                  # [web:11]

def poly_from_list(coeffs, R):
    # coeffs = [a_0, a_1, ..., a_n] with a_k the coefficient of x^k
    return sum(R(c)*x^k for k, c in enumerate(coeffs))

# example: rebuild polynomials
#polys = [poly_from_list(L, R) for L in c_polys]         # [web:11]

recovered_polys=[]
for tupl in c_polys:
    n=tupl[0]
    poly_list=tupl[1]
    poly_list.reverse()
    poly=poly_from_list(poly_list,R)
    recovered_polys.append((n,poly))
    
mins_list=[]
count=0
for tupl in recovered_polys:
    count+=1
    print(count)
    n=tupl[0]
    poly=tupl[1]
    roots=poly.roots(ring=CC, multiplicities=False)
    aar = [r.abs() for r in roots]
    if aar!=[]:
        minn = min(aar)  
        mins_list.append((n,minn))

p = list_plot(mins_list, plotjoined=True, axes_labels=['', ''])
print("MINIMUM MODULI")
p.show()

log_mins_list=[
    (pair[0],log(pair[1])) 
     for pair in mins_list]
p = list_plot(log_mins_list, plotjoined=True, axes_labels=['', ''])
print("LOG MINIMUM MODULI")
p.show()
    

n: 2
n: 3
n: 4
n: 5
n: 6
n: 7
n: 8
n: 9
n: 10
n: 11
n: 12
n: 13
n: 14
n: 15
n: 16
n: 17
n: 18
n: 19
n: 20
n: 21
n: 22
n: 23
n: 24
n: 25
n: 26
n: 27
n: 28
n: 29
n: 30
n: 31
n: 32
n: 33
n: 34
n: 35
n: 36
n: 37
n: 38
n: 39
n: 40
n: 41
n: 42
n: 43
n: 44
n: 45
n: 46
n: 47
n: 48
n: 49
n: 50
n: 51
n: 52
n: 53
n: 54
n: 55
n: 56
n: 57
n: 58
n: 59
n: 60
n: 61
n: 62
n: 63
n: 64
n: 65
n: 66
n: 67
n: 68
n: 69
n: 70
n: 71
n: 72
n: 73
n: 74
n: 75
n: 76
n: 77
n: 78
n: 79
n: 80
n: 81
n: 82
n: 83
n: 84
n: 85


KeyboardInterrupt: 

In [ ]:
# FOURIER ANALYSIS OF MINIMUM MODULI
# This cell performs spectral analysis on the minimum moduli sequence
# to detect periodicities, dominant frequencies, and oscillatory patterns

import numpy as np
from scipy import signal
import matplotlib.pyplot as plt

# ==============================================================================
# 1. PREPARE THE DATA - Convert from SageMath to Python/NumPy types
# ==============================================================================

# Extract the data from mins_list and convert to Python floats
# This is crucial because SageMath uses its own numeric types
indices = np.array([float(pair[0]) for pair in mins_list], dtype=np.float64)
min_moduli = np.array([float(pair[1]) for pair in mins_list], dtype=np.float64)

# Also prepare the log-transformed data
log_min_moduli = np.log(min_moduli)

print(f"Data range: n = {int(indices[0])} to {int(indices[-1])}")
print(f"Number of data points: {len(indices)}")
print(f"Min modulus range: {min_moduli.min():.6f} to {min_moduli.max():.6f}")

# ==============================================================================
# 2. DETRENDING (Important for seeing oscillations)
# ==============================================================================

# For the log data, remove any polynomial trend to focus on oscillations
# We'll try both linear and quadratic detrending

# Linear detrend
log_detrended_linear = signal.detrend(log_min_moduli, type='linear')

# Polynomial detrend (degree 2)
poly_coeffs = np.polyfit(indices, log_min_moduli, deg=2)
poly_trend = np.polyval(poly_coeffs, indices)
log_detrended_poly = log_min_moduli - poly_trend

print(f"\nTrend removed. Detrended data std dev: {np.std(log_detrended_poly):.4f}")

# ==============================================================================
# 3. COMPUTE FOURIER TRANSFORM
# ==============================================================================

# Use FFT on the detrended log data
# The FFT reveals which "frequencies" (periodicities in n) are present

fft_result = np.fft.fft(log_detrended_poly)
fft_frequencies = np.fft.fftfreq(len(indices), d=1.0)  # d=1 means indices are spaced by 1

# Power spectrum (magnitude squared)
power_spectrum = np.abs(fft_result)**2

# Only keep positive frequencies (negative are redundant for real signals)
positive_freq_mask = fft_frequencies > 0
frequencies = fft_frequencies[positive_freq_mask]
power = power_spectrum[positive_freq_mask]

# Convert frequency to period: period = 1/frequency
# (period tells us "spikes repeat every X indices")
periods = 1.0 / frequencies

# ==============================================================================
# 4. IDENTIFY DOMINANT FREQUENCIES
# ==============================================================================

# Find the top 10 peaks in the power spectrum
peak_indices = signal.find_peaks(power, height=np.max(power)*0.1)[0]  # peaks at least 10% of max
peak_powers = power[peak_indices]
peak_periods = periods[peak_indices]

# Sort by power (strongest first)
sorted_indices = np.argsort(peak_powers)[::-1]
top_peaks = sorted_indices[:min(10, len(sorted_indices))]

print("\n" + "="*70)
print("TOP DOMINANT PERIODICITIES (in index n)")
print("="*70)
print(f"{'Rank':<6} {'Period':<12} {'Frequency':<12} {'Power':<15}")
print("-"*70)
for rank, idx in enumerate(top_peaks, 1):
    period = peak_periods[idx]
    freq = frequencies[peak_indices[idx]]
    pwr = peak_powers[idx]
    print(f"{rank:<6} {period:>10.2f}   {freq:>10.6f}   {pwr:>12.2e}")

# ==============================================================================
# 5. VISUALIZATIONS
# ==============================================================================

fig, axes = plt.subplots(3, 2, figsize=(16, 14))

# Plot 5.1: Original data (linear scale)
axes[0, 0].plot(indices, min_moduli, 'b-', linewidth=0.8)
axes[0, 0].set_xlabel('Index n')
axes[0, 0].set_ylabel('Minimum Modulus')
axes[0, 0].set_title('Original: Minimum Moduli')
axes[0, 0].grid(True, alpha=0.3)

# Plot 5.2: Log scale data
axes[0, 1].plot(indices, log_min_moduli, 'b-', linewidth=0.8)
axes[0, 1].set_xlabel('Index n')
axes[0, 1].set_ylabel('Log(Minimum Modulus)')
axes[0, 1].set_title('Log-transformed Minimum Moduli')
axes[0, 1].grid(True, alpha=0.3)

# Plot 5.3: Detrended data (shows oscillations more clearly)
axes[1, 0].plot(indices, log_detrended_poly, 'b-', linewidth=0.8)
axes[1, 0].set_xlabel('Index n')
axes[1, 0].set_ylabel('Log(Min Modulus) - Trend')
axes[1, 0].set_title('Detrended Log Data (Oscillations Emphasized)')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)

# Plot 5.4: Power Spectrum vs Frequency
axes[1, 1].semilogy(frequencies, power, 'b-', linewidth=0.8)
axes[1, 1].semilogy(frequencies[peak_indices[top_peaks]], 
                     peak_powers[top_peaks], 'ro', markersize=8, label='Dominant peaks')
axes[1, 1].set_xlabel('Frequency (cycles per index)')
axes[1, 1].set_ylabel('Power')
axes[1, 1].set_title('Power Spectrum vs Frequency')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_xlim(0, 0.5)  # Nyquist limit

# Plot 5.5: Power Spectrum vs Period (often more intuitive)
# Only plot periods up to half the data length (longer periods unreliable)
max_period = len(indices) / 2
period_mask = periods <= max_period
axes[2, 0].semilogy(periods[period_mask], power[period_mask], 'b-', linewidth=0.8)
# Mark dominant periods
for idx in top_peaks:
    if peak_periods[idx] <= max_period:
        axes[2, 0].semilogy(peak_periods[idx], peak_powers[idx], 'ro', markersize=8)
axes[2, 0].set_xlabel('Period (indices between repetitions)')
axes[2, 0].set_ylabel('Power')
axes[2, 0].set_title('Power Spectrum vs Period')
axes[2, 0].grid(True, alpha=0.3)
axes[2, 0].set_xlim(2, max_period)

# Plot 5.6: Spectrogram (time-frequency analysis using Short-Time Fourier Transform)
# This shows if periodicities change over the sequence
window_size = min(64, len(indices)//4)  # adaptive window size
f_spec, t_spec, Sxx = signal.spectrogram(log_detrended_poly, fs=1.0, 
                                          nperseg=window_size, 
                                          noverlap=window_size//2)
# Convert frequency axis to period for easier interpretation
period_spec = 1.0 / (f_spec + 1e-10)  # avoid division by zero
period_spec = period_spec[1:]  # remove DC component
Sxx = Sxx[1:, :]

# Plot with period on y-axis (inverted so small periods at bottom)
im = axes[2, 1].pcolormesh(indices[0] + t_spec, period_spec, 10*np.log10(Sxx + 1e-10),
                           shading='gouraud', cmap='viridis')
axes[2, 1].set_xlabel('Index n')
axes[2, 1].set_ylabel('Period')
axes[2, 1].set_title('Spectrogram: How Periodicities Change with n')
axes[2, 1].set_ylim(2, min(100, max_period))
axes[2, 1].invert_yaxis()  # small periods at bottom
plt.colorbar(im, ax=axes[2, 1], label='Power (dB)')

plt.tight_layout()
plt.show()

# ==============================================================================
# 6. FILTERED SIGNAL (reconstruct using only dominant frequencies)
# ==============================================================================

# Reconstruct signal using only the top 3 dominant frequencies
# This shows what the "pure" oscillatory pattern looks like

print("\n" + "="*70)
print("SIGNAL RECONSTRUCTION")
print("="*70)

reconstructed = np.zeros_like(log_detrended_poly)
for rank, idx in enumerate(top_peaks[:3], 1):  # Use top 3 frequencies
    freq = frequencies[peak_indices[idx]]
    period = peak_periods[idx]
    # Get amplitude and phase from FFT
    amp = np.abs(fft_result[peak_indices[idx]]) / len(indices)
    phase = np.angle(fft_result[peak_indices[idx]])
    # Reconstruct this component
    component = amp * np.cos(2 * np.pi * freq * indices + phase)
    reconstructed += component
    print(f"Component {rank}: Period = {period:.2f}, Amplitude = {amp:.4f}")

# Plot comparison
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

axes[0].plot(indices, log_detrended_poly, 'b-', linewidth=0.8, alpha=0.7, label='Original (detrended)')
axes[0].plot(indices, reconstructed, 'r-', linewidth=1.5, label='Reconstructed (top 3 frequencies)')
axes[0].set_xlabel('Index n')
axes[0].set_ylabel('Log(Min Modulus) - Trend')
axes[0].set_title('Original vs Reconstructed Signal')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Residual (what's left after removing dominant frequencies)
residual = log_detrended_poly - reconstructed
axes[1].plot(indices, residual, 'g-', linewidth=0.8)
axes[1].set_xlabel('Index n')
axes[1].set_ylabel('Residual')
axes[1].set_title('Residual After Removing Top 3 Frequencies')
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='r', linestyle='--', alpha=0.5)

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("Analysis complete! Examine the plots and frequency table above.")
print("="*70)